<a href="https://colab.research.google.com/github/gmauricio-toledo/NLP-LCC/blob/main/Notebooks/11-LLM-Finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>Finetuning de LLM para clasificación </h1>

En este notebook se muestra cómo realizar *fine-tuning* (ajuste fino) de un modelo de lenguaje preentrenado utilizando Hugging Face Transformers. Este proceso te permitirá adaptar un LLM a una tarea específica de procesamiento de lenguaje natural (NLP), como clasificación de texto o análisis de sentimientos, mejorando significativamente su rendimiento en tus datos personalizados.

In [1]:
!pip install transformers datasets evaluate torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 44.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 66.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1

In [5]:
!pip install --upgrade transformers

In [6]:
import transformers
print(transformers.__version__)

4.51.3


## Dataset

In [14]:
from datasets import Dataset

data = {
    "text": [
        "Me encanta este producto, es increíble.",
        "Lo odio, fue una mala experiencia.",
        "Excelente servicio al cliente.",
        "Muy malo, no lo recomiendo.",
        "Me siento feliz con la compra.",
        "No volveré a comprar aquí.",
        "La calidad es excelente y el envío rápido.",
        "Terrible experiencia, llegó roto y sin soporte.",
        "Totalmente satisfecho, lo compraré nuevamente.",
        "No cumple con lo prometido, muy decepcionado.",
        "Buen producto en general, aunque podría mejorar el empaque.",
        "No es lo peor, pero tampoco fue una gran experiencia."
    ],
    "label": [1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0]  # 1 = positivo, 0 = negativo
}

# Convertir a Dataset de Hugging Face
dataset = Dataset.from_dict(data)

## Finetuning

In [15]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import torch
import numpy as np
from sklearn.metrics import accuracy_score, f1_score


# Usar GPU si está disponible
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Cargar modelo y tokenizador
# model_name = "Qwen/Qwen2-0.5B-Instruct"
model_name = "Qwen/Qwen3-0.6B"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Añadir pad_token si no existe (es un modelo para generar texto)
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    print("Pad token added:", tokenizer.pad_token)

# Cargar modelo y redimensionar
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2).to(device)
model.resize_token_embeddings(len(tokenizer))

# Asignar pad_token_id al modelo (es un modelo para generar texto)
model.config.pad_token_id = tokenizer.pad_token_id


# Tokenizar los textos
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

# 4. Dividir en train y test (opcional)
tokenized_datasets = tokenized_datasets.train_test_split(test_size=0.2)
train_dataset = tokenized_datasets["train"]
eval_dataset = tokenized_datasets["test"]

# 5. Definir métricas
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds)
    return {"accuracy": acc, "f1": f1}

# 6. Configurar TrainingArguments
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=3,
    weight_decay=0.01,
    save_strategy="epoch",
    logging_dir='./logs',
    logging_steps=10,
    report_to="none",
    push_to_hub=False,
)

# 7. Inicializar Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

# 8. Entrenar
trainer.train()

# 9. Evaluar
results = trainer.evaluate()
print("Evaluation results:", results)

Using device: cuda


Some weights of Qwen3ForSequenceClassification were not initialized from the model checkpoint at Qwen/Qwen3-0.6B and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/12 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,1.479853,0.666667,0.800000
2,4.104400,0.740456,0.666667,0.666667
3,4.104400,0.241333,1.000000,1.000000


Evaluation results: {'eval_loss': 0.2413334995508194, 'eval_accuracy': 1.0, 'eval_f1': 1.0, 'eval_runtime': 0.2054, 'eval_samples_per_second': 14.604, 'eval_steps_per_second': 9.736, 'epoch': 3.0}


In [3]:
import torch
import torch.nn.functional as F

def predict_sentiment(text):
    # Tokenizar entrada
    inputs = tokenizer(text, padding=True, truncation=True, max_length=128, return_tensors="pt").to(device)

    # Hacer inferencia sin calcular gradientes
    with torch.no_grad():
        outputs = model(**inputs)

    # Obtener logits y aplicar softmax para probabilidades
    logits = outputs.logits
    probs = F.softmax(logits, dim=-1).cpu().numpy()[0]

    # Decodificar resultado
    predicted_class = logits.argmax(dim=-1).item()
    label_map = {0: "Negativo", 1: "Positivo"}

    return {
        "texto": text,
        "clase": label_map[predicted_class],
        "probabilidad_positiva": probs[1],
        "probabilidad_negativa": probs[0]
    }

In [10]:
# Algunos ejemplos de frases
examples = [
    "Este producto es increíble, lo amo.",
    "No volveré a comprar aquí, muy mala experiencia.",
    "Excelente atención al cliente.",
    "Es una basura, no lo recomiendo.",
    "Es malísimo, no cumple con lo prometido.",
    "Me siento feliz con mi decisión.",
]

# Hacer predicciones
for example in examples:
    result = predict_sentiment(example)
    print(f"Texto: {result['texto']}")
    print(f"Predicción: {result['clase']}")
    print(f"Probabilidad positiva: {result['probabilidad_positiva']:.4f}")
    print("-"*50)

Texto: Este producto es increíble, lo amo.
Predicción: Positivo
Probabilidad positiva: 0.9074
--------------------------------------------------
Texto: No volveré a comprar aquí, muy mala experiencia.
Predicción: Negativo
Probabilidad positiva: 0.0123
--------------------------------------------------
Texto: Excelente atención al cliente.
Predicción: Positivo
Probabilidad positiva: 0.7898
--------------------------------------------------
Texto: Es una basura, no lo recomiendo.
Predicción: Negativo
Probabilidad positiva: 0.2614
--------------------------------------------------
Texto: Es malísimo, no cumple con lo prometido.
Predicción: Negativo
Probabilidad positiva: 0.2503
--------------------------------------------------
Texto: Me siento feliz con mi decisión.
Predicción: Positivo
Probabilidad positiva: 0.9989
--------------------------------------------------


In [5]:
import torch
import torch.nn.functional as F

def get_embedding(text):
    # Tokenizar texto
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=128).to(device)

    # Hacer inferencia sin gradientes
    with torch.no_grad():
        outputs = model.base_model(**inputs)  # base_model es el modelo sin la capa de clasificación

    # Obtener las salidas del últimos estados ocultos (last_hidden_state)
    last_hidden_states = outputs.last_hidden_state  # Forma: [batch_size, seq_length, hidden_size]

    # Promediar embeddings de tokens que son padding
    mask = inputs['attention_mask'].unsqueeze(-1)
    masked_embeddings = last_hidden_states * mask
    summed = torch.sum(masked_embeddings, dim=1)
    counts = torch.clamp(torch.sum(mask, dim=1), min=1e-9)
    mean_pooled = summed / counts  # Forma: [1, hidden_size]

    return mean_pooled.cpu().numpy().flatten()

In [13]:
texts = [
    "Este producto es excelente.",
    "Estoy feliz con mi compra.",
    "No me gusto el empaqué, la calidad no es muy buena y el precio es alto para lo que es",
    "Se ve de decente calidad"
]

# Matriz de embeddings:
X = [get_embedding(text) for text in texts]
X = np.array(X)
print(X.shape)

# Exploración de algunos embeddings:
for text in texts[:3]:
    embedding = get_embedding(text)
    print(f"Texto: {text}")
    print(f"Embedding shape: {embedding.shape}")
    print(embedding[:5])
    print("-"*80)

(4, 1024)
Texto: Este producto es excelente.
Embedding shape: (1024,)
[ 1.6442688  9.2536955 -1.031303   6.623998   1.2147993]
--------------------------------------------------------------------------------
Texto: Estoy feliz con mi compra.
Embedding shape: (1024,)
[ 1.899242   8.043665  -1.1767625 10.03683    2.145203 ]
--------------------------------------------------------------------------------
Texto: No me gusto el empaqué, la calidad no es muy buena y el precio es alto para lo que es
Embedding shape: (1024,)
[ 0.53470224 -6.5924377  -1.3666779  11.78304     1.8274125 ]
--------------------------------------------------------------------------------
